<a href="https://colab.research.google.com/github/elchaima1234/Data-Preprocessing/blob/main/Audio_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install pesq

  Preparing metadata (setup.py) ... done
  Created wheel for pesq: filename=pesq-0.0.4-cp310-cp310-linux_x86_64.whl size=262949 sha256=f284e7e0e497a1d5019407dda70ef934cf17577ac5800051a8ccf96a4046afcf
  Stored in directory: /root/.cache/pip/wheels/c5/4e/2c/251524370c0fdd659e99639a0fbd0ca5a782c3aafcd456b28d
Successfully built pesq


In [19]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 70.4 MB/s eta 0:00:00


In [24]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import soundfile as sf
from scipy.io import wavfile
import tensorflow as tf
import os
from pesq import pesq
import pywt

#**STFT**

In [10]:
audio_data = '/content/SA1.WAV.wav'
x , _ = librosa.load(audio_data,sr=16000)
ipd.Audio(audio_data)

In [15]:
sr=16000
S = librosa.stft(x, n_fft=1024,hop_length=512, window='hann')
inv = librosa.istft(S, n_fft=1024,hop_length=512, window='hann')
sf.write('inverse.wav', inv , samplerate=sr)

In [16]:
rate, ref = wavfile.read("/content/SA1.WAV.wav")
rate, deg = wavfile.read("/content/inverse.wav")

In [17]:
print(pesq(rate, ref, deg, 'wb'))
#print(pesq(rate, ref, deg, 'nb'))

4.643573760986328


#**DWT**

In [21]:
samplerate, data = wavfile.read('/content/SA1.WAV.wav');  # Reading the audio file
t = np.arange(len(data)) / float(samplerate);  # Retrieving Time

print("data -> ", data)
print("data length -> ", len(data))
print("np.arange(len(data)) -> ", np.arange(len(data)))
print("float(samplerate) -> ", float(samplerate))
print("time -> ", t)

data = data/max(data);  # Normalize Audio Data
print("normalized data -> ", data)

data ->  [ 3  5  2 ...  8 10  7]
data length ->  63488
np.arange(len(data)) ->  [    0     1     2 ... 63485 63486 63487]
float(samplerate) ->  16000.0
time ->  [0.0000000e+00 6.2500000e-05 1.2500000e-04 ... 3.9678125e+00 3.9678750e+00
 3.9679375e+00]
normalized data ->  [0.00038911 0.00064851 0.0002594  ... 0.00103761 0.00129702 0.00090791]


In [22]:
samplerate =16000
ca,cd =pywt.dwt(data, "bior6.8", "per")
y=pywt.idwt(ca, cd, "bior6.8", "per")
wavfile.write('sampleR.wav', samplerate, y);  # writing y as Audio
wavfile.write('sampleA.wav', samplerate, ca);  # writing ca as Audio
wavfile.write('sampleD.wav', samplerate, cd);  # writing cd as Audio

# **Mel-spectrogram**

In [25]:
# parameters
sr = 22050
n_mels = 128
hop_length = 512
n_iter = 32
n_mfcc = 13 # can try n_mfcc=20

# load audio and create Mel-spectrogram
path = '/content/SA1.WAV.wav'
y, _ = librosa.load(path, sr=sr)
S = np.abs(librosa.stft(y, hop_length=hop_length, n_fft=hop_length*2))
mel_spec = librosa.feature.melspectrogram(S=S, sr=sr, n_mels=n_mels, hop_length=hop_length)

# optional, compute MFCCs in addition
if n_mfcc is not None:
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(S), sr=sr, n_mfcc=n_mfcc)
    mel_spec = librosa.feature.inverse.mfcc_to_mel(mfcc, n_mels=n_mels)

# Invert mel-spectrogram
S_inv = librosa.feature.inverse.mel_to_stft(mel_spec, sr=sr, n_fft=hop_length*4)
y_inv = librosa.griffinlim(S_inv, n_iter=n_iter,
                            hop_length=hop_length)

soundfile.write('orig.wav', y, samplerate=sr)
soundfile.write('inv.wav', y_inv, samplerate=sr)

#**spectrogram**

In [29]:
audio_dir='/content/'
audio_list=[]
spectrogram=[]

for file_name in os.listdir(audio_dir):
  if file_name.endswith('.wav'):
    file_path = os.path.join(audio_dir, file_name)
    y,sr = librosa.load(file_path, sr=16000) # y -> array time series (my_audio as np.array)
    spec = librosa.feature.melspectrogram(y=y,
                                          sr=sr,
                                          n_fft=1024,
                                          hop_length=512,
                                          win_length=None,
                                          window='hann',
                                          center=True,
                                          pad_mode='reflect',
                                          power=2.0,
                                          n_mels=128)

    audio_list.append(y)
    spectrogram.append(spec )


In [30]:
spectrogram[5].shape

(128, 125)

# **mel spectrogram**

In [31]:
my_sample_rate = 16000
# step1 - converting a wav file to numpy array and then converting that to mel-spectrogram
my_audio_as_np_array, my_sample_rate= librosa.load("/content/SA1.WAV.wav")

# step2 - converting audio np array to spectrogram
spec = librosa.feature.melspectrogram(y=my_audio_as_np_array,
                                        sr=my_sample_rate,
                                            n_fft=1024,
                                            hop_length=512,
                                            win_length=None,
                                            window='hann',
                                            center=True,
                                            pad_mode='reflect',
                                            power=2.0)
                                            #n_mels=128)

# step3 converting mel-spectrogrma back to wav file

res = librosa.feature.inverse.mel_to_audio(spec,
                                           sr=my_sample_rate,
                                           n_fft=1024,
                                           hop_length=512,
                                           win_length=None,
                                           window='hann',
                                           center=True,
                                           pad_mode='reflect',
                                           power=2.0,
                                           n_iter=32)
                                           #n_mels=128)

# step4 - save it as a wav file
import soundfile as sf
sf.write("test2.wav", res, my_sample_rate)

In [32]:
my_sample_rate = 16000
# step1 - converting a wav file to numpy array and then converting that to mel-spectrogram
my_audio_as_np_array, my_sample_rate= librosa.load("/content/SA1.WAV.wav")
# step2 - converting audio np array to spectrogram
spec = librosa.feature.melspectrogram(y=my_audio_as_np_array,
                                        sr=my_sample_rate,
                                            n_fft=1048,
                                            hop_length=512,
                                            win_length=None,
                                            window='hann',
                                            center=True,
                                            pad_mode='reflect',
                                            power=2.0,
                                            n_mels=128)

# step3 converting mel-spectrogrma back to wav file

res = librosa.feature.inverse.mel_to_stft(spec,
                                           sr=my_sample_rate,
                                           n_fft=1048)



y = librosa.griffinlim(res,
                       n_fft=1048,
                      hop_length=512,
                      win_length=None,
                      window='hann',
                      center=True,
                      pad_mode='reflect')


# step4 - save it as a wav file
import soundfile as sf
sf.write("test1.wav", y, my_sample_rate)

# **mfcc**

In [33]:
sr = 16000
n_fft = 1024
hop_length = 512

In [34]:
mfccs = librosa.feature.mfcc(y=my_audio_as_np_array,
                             sr=sr,
                             n_fft=n_fft,
                             hop_length=hop_length,
                             n_mfcc=500)
mfcc_inver = librosa.feature.inverse.mfcc_to_audio(mfccs,
                                                   sr=sr,
                                                   n_fft=n_fft,
                                                   hop_length=hop_length,
                                                   )


import soundfile as sf
sf.write("test1.wav", mfcc_inver, sr)